In [ ]:
#!pip3 install datasets

In [ ]:
#!pip3 install transformers -U;

In [ ]:
#!pip3 install accelerate -U

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import torch
import tensorflow as tf

from sklearn.model_selection import train_test_split

import datasets
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
# !git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

In [ ]:
# Get dataset for working locally
df_train = pd.read_csv("drugsTrain_processed_for_Model_3.csv", sep="\t")
df_test = pd.read_csv("drugsTest_processed_for_Model_3.csv", sep="\t")

In [ ]:
# Get dataset when working on colab
# df_train = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTrain_processed_for_Model_3.csv", sep="\t")
# df_test = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTest_processed_for_Model_3.csv", sep="\t")

In [ ]:
df_train = df_train.sample(frac=0.3, random_state=42).reset_index()
df_test = df_test.sample(frac=0.3, random_state=42).reset_index()

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_val, df_test_new = train_test_split(df_test, test_size=0.6, random_state=42)

In [ ]:
ds_train = Dataset.from_pandas(df_train[["processed_review", "rating"]])
ds_val = Dataset.from_pandas(df_val.reset_index()[["processed_review", "rating"]])
ds_test = Dataset.from_pandas(df_test_new.reset_index()[["processed_review", "rating"]])

In [ ]:
ds = DatasetDict()

ds["train"] = ds_train
ds["validation"] = ds_val
ds["test"] = ds_test

In [ ]:
ds

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def preprocess(samples):
  text = samples["processed_review"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_attention_mask=True)
  encoding["labels"] = samples["rating"]
  return encoding

In [ ]:
encoded_dataset = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

In [ ]:

args = TrainingArguments(
    output_dir="../bert-regression",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  mse = mean_squared_error(labels, predictions)
  return {"mse": mse}

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("./bert-regression")

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained("./bert-regression/checkpoint-250")

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["validation"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [ ]:
trainer.evaluate(eval_dataset=encoded_dataset["test"])